# Deliverable 1. Retrieve Weather Data.
---
1. Create a folder called `Weather_Database` to save all the files related with this deliverable.

2. Save the `Weather_Database_starter_code.ipynb` starter code to the `Weather_Database` folder and rename it as `Weather_Database.ipynb`.

3. Use the `np.random.uniform` function to generate a new set of 2,000 random latitudes and 2,000 longitudes.

4. Use the `citipy` module to get the nearest city for each latitude and longitude combination.

5. Import your OpenWeatherMap's API key and assemble the API call URL as a string variable. Recall to edit the `config.py` file to add your API key; also, it's critical to avoid publishing your API key on your GitHub repository.

6. Retrieve the following information from the API call:

    * Latitude and longitude

    * Maximum temperature

    * Percent humidity

    * Percent cloudiness

    * Wind speed

    * Weather description (for example, clouds, fog, light rain, clear sky)

7. Add the weather data to a new DataFrame.

8. Export the DataFrame as a CSV file, and save it as `WeatherPy_Database.csv` in the `Weather_Database` folder.
---

In [1]:
# Import initial libraries
import numpy as np
import pandas as pd
from citipy import citipy

## Use the `np.random.uniform` function to generate a new set of 2,000 random latitudes and 2,000 longitudes.

In [2]:
# Create a set of random latitude and longitude combinations
lats = np.random.uniform(-90,90,2)
lngs = np.random.uniform(-180,180,2)

# Use the zip function to create an iterator of tuples containing the latitude and longitude combinations
lat_lngs = zip(lats,lngs)


In [4]:
# Add the latitudes and longitudes to a list
coordinates=list(lat_lngs)
print(coordinates[:5])

[(-61.66881091051005, -42.38622180828759), (3.9163892408921726, 102.8738572703308)]


## Use the `citipy` module to get the nearest city for each latitude and longitude combination.

In [5]:
# Create an empty list for holding the cities
cities = []

# Use a for loop to identify nearest city for each latitude and longitude combination using the citipy module
for coordinate in coordinates:
    # Use the citipy module to identify the nearest city, country to the current coordinate
    city = citipy.nearest_city(coordinate[0],coordinate[1]) 
    #print(type(city))
    #print(city.city_name, city.country_code)
    # If the city is unique, then add it to a our cities list
    if city.city_name not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
print(len(cities))

2


## Import your OpenWeatherMap's API key and assemble the API call URL as a string variable. Recall to edit the `config.py` file to add your API key.

In [6]:
# Import the requests library
import requests

# Import the time library
import time

# Import json
import json

# Import the datetime module from the datetime library
from datetime import datetime

# Import the OpenWeatherMap's API key
from config import weather_api_key

In [7]:
# Assemble the OpenWeatherMap's API call
# url = "https://api.openweathermap.org/data/2.5/weather?q=London,uk&APPID="+weather_api_key
url = "https://api.openweathermap.org/data/2.5/weather?&APPID="+weather_api_key

## Retrieve the following information from the API call.

- The latitude and longitude
- The Max temperature
- The % humidity
- The % cloudiness
- The Wind speed
- The Weather description, i.e., cloudy, fog, light rain, clear sky, etc. 

In [8]:
# Create an empty list to hold weather data for each city
city_data = []

# Print a message to indicate that the data retrieval starts
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters and set them to 1
record_count = 1
set_count = 1

# Loop through all the cities in our list to fetch weather data for each city
# set loop counter to 0
i = 0 
for city in cities:      
    # Group cities in sets of 50 for logging purposes
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        time.sleep(60)
   
    # Create an endpoint URL for each city
    city_url = url + "&q=" + city.city_name.replace(" ","+") + "," + city.country_code
    print(city_url)
    
    # Log the url, record, and set numbers
    print(f"Processing Record {record_count} of Set {set_count} | {city.city_name},{city.country_code}")

    # Add 1 to the record count
    record_count += 1

    # Run an API request for each of the cities
    try:
        city_weather=requests.get(city_url).json()
    
        if city_weather["cod"]==200:
            # success!
            city_weather_formatted_str = json.dumps(city_weather, indent=4)
            #print(city_weather_formatted_str)
       
            # Parse out the latitude, longitude, max temp, humidity, cloudiness, wind, country, and weather description       
            long=round(city_weather['coord']['lon'],2)
            lat=round(city_weather['coord']['lat'],2)
            max_temp=city_weather['main']['temp_max']
            humidity=city_weather['main']['humidity']
            cloudiness=city_weather['clouds']['all']
            wind_speed=city_weather['wind']['speed']
            country=city_weather['sys']['country']
            city_name=city.city_name.title()
            description=city_weather["weather"][0]["description"]
            city_datum={'Lat':lat,'Lng':long,'Max Temp':max_temp,'Humidity':humidity,'Cloudiness':cloudiness,'Wind Speed':wind_speed,'City':city_name,'Country':country,'Current Description':description}
            print(city_datum)
            # Append the city information into the city_data list
            city_data.append(city_datum)
        else:
            print("HTML Error: "+city_weather["cod"]+" Message: "+city_weather["message"])
            
   
    except Exception as error:
        # If an exception is encountered print error
        print("Exception encountered.")
        print(error)
        pass
    
    # increment loop counter
    i+=1 

# Indicate that the data retrieval is complete 
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
https://api.openweathermap.org/data/2.5/weather?&APPID=492a5a3e5474d1017570fc55a90cc59e&q=ushuaia,ar
Processing Record 1 of Set 1 | ushuaia,ar
{'Lat': -54.8, 'Lng': -68.3, 'Max Temp': 277.96, 'Humidity': 75, 'Cloudiness': 40, 'Wind Speed': 4.63, 'City': 'Ushuaia', 'Country': 'AR', 'Current Description': 'light rain'}
https://api.openweathermap.org/data/2.5/weather?&APPID=492a5a3e5474d1017570fc55a90cc59e&q=kuantan,my
Processing Record 2 of Set 1 | kuantan,my
{'Lat': 3.81, 'Lng': 103.33, 'Max Temp': 299.21, 'Humidity': 83, 'Cloudiness': 20, 'Wind Speed': 3.09, 'City': 'Kuantan', 'Country': 'MY', 'Current Description': 'few clouds'}
-----------------------------
Data Retrieval Complete      
-----------------------------


In [103]:
# Print the length of the city_data list to verify how many cities you have
len(city_data)

2

## Add the weather data to a new DataFrame.

In [9]:
# Use the city_data list to create a new pandas DataFrame.
city_data_df = pd.DataFrame(city_data)

In [10]:
# Display sample data
city_data_df.head(10)

,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,City,Country,Current Description
0,-54.80,-68.30,277.96,75,40,4.63,Ushuaia,AR,light rain
1,3.81,103.33,299.21,83,20,3.09,Kuantan,MY,few clouds


In [11]:
# Display the DataFrame's column names using the columns Pandas function
city_data_df.columns

Index(['Lat', 'Lng', 'Max Temp', 'Humidity', 'Cloudiness', 'Wind Speed',
       'City', 'Country', 'Current Description'],
      dtype='object')

In [12]:
# Create a list to reorder the column names as follows:
# "City", "Country", "Lat", "Lng", "Max Temp", "Humidity",  "Cloudiness", "Wind Speed",  "Current Description"
col_names=["City", "Country", "Lat", "Lng", "Max Temp", "Humidity",  "Cloudiness", "Wind Speed",  "Current Description"]

# Recreate the DataFrame by using the new column order
city_data_df_final=city_data_df[col_names]

# Display sample data
city_data_df_final.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Ushuaia,AR,-54.80,-68.30,277.96,75,40,4.63,light rain
1,Kuantan,MY,3.81,103.33,299.21,83,20,3.09,few clouds


In [13]:
# Display the data types of each column by using the dtypes Pandas function
city_data_df_final.dtypes

City                    object
Country                 object
Lat                    float64
Lng                    float64
Max Temp               float64
Humidity                 int64
Cloudiness               int64
Wind Speed             float64
Current Description     object
dtype: object

## Export the DataFrame as a CSV file, and save it as `WeatherPy_Database.csv` in the `Weather_Database` folder.

In [14]:
# Set the output file name
filename='WeatherPy_Database.csv'

# Export the city_data DataFrame into a CSV file
city_data_df_final.to_csv(filename)